<a href="https://colab.research.google.com/github/kgoossens1/pmx/blob/master/drugsimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Classes

class Ligand: 
    """Store ligand SMILES and RDKit molecule object. Allows conversion from
    simple text file, csv file, sdf, pdb and mol2 format.0
    """

    def __init__(self, smiles):
        self.molecule = Chem.MolFromSmiles(smiles)
        self.smiles = smiles
        self.activity = float("NaN")
        return 
#If all functions fail, molecule should not be stored under self.Molecule

    def set_properties(self, property_dict, activity=False):
        """ki and IC50 values should be in micromolar units. If this is not the 
        case, calculations based on the activity will lead to wrong results.
        true name is stored under truename attribute if present.
        """
#Also possible to leave a free activity flag, in which case measurements wouldn't be converted. 
        if activity:
          print("NOTE: You have chosen to parse activity measurements. Keep in"\
                "mind that for correct results, the activities should all be"\
                "derived from the same measurement and have the same units.")
        self._data = property_dict
        if "name" in property_dict:
            self.truename = property_dict["name"]
        if activity == "ic50":
            try:
                self.activity = property_dict[activity].split()[0]
            except IndexError:
                pass
        elif activity == "pic50":
            try:
                self.activity = (
                    10**-(float(property_dict[activity].split()[0]))*10**6)
                print("pIC50 value converted to IC50 in micromolars.")
            except IndexError:
                pass
        return             
#how to better handle nm/um/pm formatting? in title/with value

    def set_properties_sdf(self, activity ="ic50"):
        """Writes the properties of the molecule to the RDKit molecule object
        so they can be parsed to output SDF files. If no activity is set, an
        empty column with header "ic50" will be written.
        """
        self.SetProp("identifier", self.name)
        if self.truename:
            self.SetProp("name", self.truename)
        self.SetProp("smiles", self.smiles)    
        self.SetProp(activity, self.activity)
        for property in ["morgan", "atompair", "rdkit", "toptors", "maccs"]:
            if self.prop:
                self.SetProp(prop, self.prop) 
        if self._data:           
            for key, item in self._data:
                self.SetProp(key, item)

        

    def gen_confs(self, nconfs=100):
        """Generates and stores conformations in self.confs"""
        print(f"Generating conformations for {self}...")
        mol = Chem.AddHs(self)
        self.confs = AllChem.EmbedMultipleConfs(
            mol, numConfs=nconfs, params=AllChem.ETKDG())
        return(f"{nconfs} conformations generated and stored to {self}.confs.")
 

class LigandSet:
    """Creates a ligand set object containing multiple ligands, stored as a list
    of molecule objects in the "mols" attribute.
    """ 

    def __init__(self, molset):
        self.mols = molset
        return
    
    def add_mols(self, *args):
        for mol in args:
            self.mols.append(mol)
        return
    
    def gen_fp(self, FPtype='morgan', **kwargs):
        """Generates fingerprints for all ligands in the ligand set. Options are
        morgan, atompair, rdkit, toptors, maccs or all. 
        Optional to generate fingerprints with other parameters than the default
        by using keyword arguments.
        """ 
        morgan = rdFingerprintGenerator.GetMorganGenerator(**kwargs)
        atompair = rdFingerprintGenerator.GetAtomPairGenerator(**kwargs)
        toptors = rdFingerprintGenerator.GetTopologicalTorsianGenerator(**kwargs)
        rdkit = rdFingerprintGenerator.GetRDKitFPGenerator(**kwargs)
        for mol in self.ligands:
            if FPtype == 'all':
                for fp in ["morgan", "atompair", "rdkit", "toptors"]:
                    mol.fp = fp.GetFingerprint(mol)     #possible naming issue with "fp" as variable and in list
                mol.maccs = MACCSkeys.GenMACCSKeys(mol, **kwargs)
            elif FPtype == 'MACCS':
                mol.maccs = MACCSkeys.GenMACCSKeys(mol, **kwargs)
            else:
                mol.FPtype = FPtype.GetFingerprint(mol)
        return(f"{FPtype} fingerprints were set for all molecules in ligand set.")
            

In [ ]:
#readers

import os
import csv
import math
import shlex
import subprocess as sp

#from ds_classes import *
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, MACCSkeys, rdFMCS, rdFingerprintGenerator

import pandas as pd

def mols_from_csv(csv_data, activity=False):
    """Extract multiple ligands from CSV file format. Make sure a header row is
    present containing the names of the fields. The header for the molecule in
    SMILES format should be 'smiles'. If activity data is present, the header
    should be "ic50", "pic50" or "ki", corresponding to the measurement.
    """

    cwd = os.getcwd()
    csvpath = os.path.join(cwd, csv_data)
    with open(csvpath, newline = '') as csvfile:
        reader = csv.DictReader(csvfile)
        for count, row in enumerate(reader, start=1):
            lig = "ligand"+str(count)
            ligname = lig
            lig = Ligand(row["smiles"])
            lig.name = ligname 
            lig.set_properties(row,activity)
            mol_list.append(lig)
    
    return(f"{count} molecules converted from CSV format.")

def mols_from_sdf(sdf_data, activity=False):
    """Extract multiple ligands from SDF file, including the properties present
    in the property fields.
    """

    cwd = os.getcwd()
    sdfpath = os.path.join(cwd, sdf_data)
    mols = Chem.SDMolSupplier(sdfpath)
    for count, mol in enumerate(mols, start=1):
        lig = "ligand"+str(count)
        ligname = lig
        property_dict = mol.GetPropsAsDict() 
        if "smiles" in property_dict:
            lig = Ligand(property_dict["smiles"])  
        else:
            smiles_string = Chem.MolToSmiles(mol)
            lig = Ligand(smiles_string)
        lig.molecule = mol
        lig.set_properties(property_dict,activity)
        lig.name = ligname 
        mol_list.append(lig)
    return(f"{count} molecules converted from SDF format.")
                  
def mols_from_smi(smi_file, activity=False, *args):        
    """If SMILES file has no additional data, don't use any additional arguments.
    If other data exists, use the use the preferred propertynames as arguments
    in the correct order. Activity remains as a kwarg (ic50 or pic50 or ki). The 
    input file should also not contain a header row.
    """
    
    with open(smi_file, 'r') as f:
        property_dict={}
        for count, line in enumerate(f, start=1):
            lig = "ligand"+str(count)
            ligname = lig 
            lig = Ligand(line.split()[0])
            lig.name = ligname
            if args:
                for count, arg in enumerate(args, start=1):
                    property_dict[arg]=line.split()[count]
                    print(property_dict)
                    lig.set_properties(property_dict,activity)            
            lig.name = ligname    
            mol_list.append(lig)
        return(f"{count} molecules converted from SMILES format.")
            
def mols_from_pdb(*args):
    """"No data is stored in PDB files aside from structure. Only accepts PDB
    files with single molecules.
    """

    print("WARNING: Using he PDB file format is not recommended for"\
          "cheminformatics. Please check the output datn ma to make sure no errors"\
          "have been made in connectivity and charge.")
    failedcounter = 0
    for count, pdbfile in enumerate(args, start=1):
        try:
            mol=Chem.MolFromPDBFile(pdbfile)
            smiles=Chem.MolToSmiles(mol)
            lig = "ligand"+str(count)
            ligname = lig 
            lig = Ligand(smiles)
            lig.name = ligname    
            mol_list.append(lig)
        except Exception as e:
            failedcounter += 1
            print(e, f"\nFailed when parsing {ligname}.")
            continue
        
    return (f"{count-failedcounter} molecules converted from PDB format."\
            f"{failedcounter} molecules didn't get converted.")

def mols_from_mol2(*args):
    """"Property data in MOL2 files is not extracted.
    """

    print("WARNING: Using the MOL2 file format is not recommended for cheminformatics."\
          "Please check the output data to make sure no errors have been made"\
          "in connectivity and charge.")
    failedcounter = 0
    for count, mol2file in enumerate(args, start=1):
        try:
            mol=Chem.MolFromMol2File(mol2file)
            smiles=Chem.MolToSmiles(mol)
            lig = "ligand"+str(count)
            ligname = lig 
            lig = Ligand(smiles)
            lig.name = ligname    
            mol_list.append(lig)
        except Exception as e:
            failedcounter += 1
            print(e, f"\nFailed when parsing {ligname}.")
            continue
        
    return (f"{count-failedcounter} molecules converted from MOL2 format."\
            f"{failedcounter} molecules didn't get converted.")


In [ ]:
#utils

def get_fp_info():
    """Print info strings given for the different fingerprint generators."""

    print("Morgan fingerprints:")
    rdFingerprintGenerator.GetMorganGenerator().GetInfoString()
    print("AtomPair fingerprints:")
    rdFingerprintGenerator.GetAtomPairGenerator().GetInfoString()
    print("Topological Torsion fingerprints:")
    rdFingerprintGenerator.GetTopologicalTorsianGenerator().GetInfoString()
    print("RDKit fingerprints:")
    rdFingerprintGenerator.GetRDKitFPGenerator().GetInfoString()

def find_MCS(ligandset):
    """Expects list of mol objects. Finds the maximum common substructure and
    writes it to self.mcs if input is a LigandSet object. If not, returns MCS.
    """

    substructure = rdFMCS.FindMCS(ligandset, ringMatchesRingOnly=True)
    try:
        ligandset.mcs = substructure
        print(f"Maximum common structure with {substructure.numAtoms()} atoms "\
              f"found and stored in '{ligandset}.mcs'.")
    except AttributeError:
        print(f"Maximum common structure with {substructure.numAtoms()} atoms found.")
        return substructure
    return

def get_best_conf(mol, maxits=100):
    """Minimizes the conformation that is lowest in energy and stores key and
    resulting conformation in a list under self.minconf."""
    confenergies = {}
    for conf in mol.confs:
        ff =AllChem.MMFFGetMoleculeForceField(
            mol, AllChem.MMFFGetMoleculeProperties(mol), confId=conf)
        ff.Initialize()
        confenergies[conf] = ff.CalcEnergy()
    lowest_energy = min(confenergies.values())
    LE_key = [key for key in confenergies if confenergies[key] == lowest_energy]
    ff =AllChem.MMFFGetMoleculeForceField(
            mol, AllChem.MMFFGetMoleculeProperties(mol), confId=LE_key)
    ff.Initialize()
    ff.CalcEnergy()
    mol.minconf = [LE_key, ff.Minimize(maxIts=maxits)]
    ff.CalcEnergy()
    return(f"Conformation {mol.confs[LE_key]} was minimized and stored in\
            '{mol}.minconf'")
#Need to test behavior. Does minimize generate conf that is saved in the list or does it update the existing conf? If so, only need to save the key and reference it in other funcs

def align_molconfs(mols, out='aligned_mols.sdf'):
    """Aligns best conformation of each molecule. First molecule in list is 
    automatically taken as the reference molecule. Result is written to an SDF
    file.
    """
    writer = Chem.SDWriter(out)
    writer.write(mols[0], mols[0].minconf[0])
    for mol in mols[1:]:
        rdMolAlign.AlignMol(mols[0], mol, mols[0].minconf[0], mol.minconf[0])
        writer.write(mol, confId=mol.minconf[0])
    return(f"{len(mols)} molecules aligned and written to '{out}''.")
#Need to test behavior. Does minimize generate conf that is saved in the list or does it update the existing conf? If so, only need to save the key and reference it in other funcs

def align_mols(mols, out='aligned_mols.sdf'):
    """Aligns molecules when no conformation generation was done. First molecule
    in list is automatically taken as the reference molecule. Result is written
    to an SDF
    file.
    """
    writer = Chem.SDWriter(out)
    writer.write(mols[0])
    for mol in mols[1:]:
        rdMolAlign.AlignMol(mols[0], mol)
        writer.write(mol)
    return(f"{len(mols)} molecules aligned and written to '{out}''.")

def write_to_sdf(input, output):
    """Separate function to write a mol object or list of mol objects to an 
    SDF file.
    """
    writer = Chem.SDWriter(output)
    writer.write(input)
    return(f"{input} written to '{output}'")

def reader(filetype, files, activity=False, ligsetname ='ligandset', *args):
    """recognizes filetype and uses appropriate function to read files. Use
    *args to define property headers for SMILES input."""

    mol_list = []
    if isinstance(files,str):
        if filetype == 'smi' or filetype == 'txt':
            mols_from_smi(files, activity, *args)
        elif filetype == 'pdb':
            mols_from_pdb(files)
        elif filetype == 'mol2':
            mols_from_mol2(files)
        elif filetype == 'sdf':
            mols_from_sdf(files, activity)       
        elif filetype == 'csv':
            mols_from_csv(files, activity)
        print(
            f"File processed and molecule(s) written to ligand set '{ligsetname}'.")

    elif isinstance(files,list):
        if (filetype == 'smi' or filetype == 'txt':
            for file in files:
                mols_from_smi(file, activity, *args)
        ligsetname = LigandSet(mol_list)       
        print(
            f"{len(files)} file(s) processed and molecules written to ligand set '{ligsetname}'.")
    else:
        print(f"Unrecognized input '{files}'. Expected string or list.")
    return


#Screen dataset for smarts query and add all molecules with query match to new ligandset (e.g. from MCS or input by user)
#Align molecules in dataset and output to sdf file (def align and use in def get_aligned)
#get optimal conformation of each molecule(calc energy), align these and output to sdf
#output conf lib for each molecule to sdf file 
#Create diverse subset from ligandset
#dataset similarity metrics and map

#converter.to_rdkit_mol

In [ ]:
import argparse



def align_for_pharmacophore(filetype, *args, outputfile, activity):
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-fi',
                        '--filetype',
                        metavar='INPUT FILE TYPE',
                        type=str,
                        required=True,
                        help='The input file type. can be csv, txt, smi, pdb,\
                             mol2 or sdf.')
    parser.add_argument('-f',
                        '--file',
                        metavar='INPUT FILE(s)',
                        type=str,
                        nargs='+',
                        required=True,
                        help='The input file(s).'
#test this parser for multiple inputfiles (for list or *). Possible fix is split() into list of args
    parser.add_argument('-o',
                        '--out',
                        metavar='OUTPUT',
                        type=str,
                        required=True,
                        help='The output file name (always with sdf suffix).')
    parser.add_argument('-a',
                        '--activity',
                        metavar='ACTIVITY',
                        type=str,
                        help='Set if activity metric is present (ic50, pic50\
                             or ki header.')
    parser.add_argument('-p',
                        '--properties',
                        metavar='PROPERTIES',
                        type=str,
                        nargs='+',
                        help='Set if activity metric is present (ic50, pic50\
                             or ki header.')
#test this parser for multiple props (for list or *). Possible fix is split() into list of args
    args = parser.parse_args()


In [ ]:

mol_list=[]
#ligands_from_smi('ROCKS-smiles.txt', activity=False)
ligands_from_csv("ROCK-inhibitors.csv", activity="ic50")
#ligands_from_sdf('ROCKS-set.sdf')
print(f"{len(mol_list)} molecules have been parsed to mol_list.")   
print(mol_list[4].name)
print(mol_list[4].smiles)
print(mol_list[4].activity)


20 molecules have been parsed to mol_list.
ligand5
CN(C)C(=O)Oc1cccc(NC(=O)C2(CN)CCN(CC2)c3ncnc4[nH]cc(C)c34)c1
nan
